In [ ]:
#Hello Professor Brooks and Sohaib!!!! Just change this and run each cell!!!!!!!!!!
option = 1 #PWR = 1, BWR = 2

In [ ]:
#imports needed to run
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from numpy import inf
from pyXSteam.XSteam import XSteam


#setting properties for each reactor
if option == 1:
    H = 4 #m
    Drod = 0.95/100 # cm to m 
    pitch = 1.26/100 # cm to m
    DFuel = 0.82/100# cm to m
    gap_t = 0.006/100 #cm to m
    k_gap = 0.25 #W/mC
    k_fuel = 3.6 #W/mC
    k_clad = 21.5 #W/mC
    G = 4000 #kg/m2s
    qo = 430*100 #W/m
    Pz0 = 15 #MPa
    Tfz0 = 277 #C
    Tf0 = Tfz0+273.15
    g = 9.81 #m/s2
    mu = 97.575e-6
    mul = 69.403e-6
    muv = 22.716e-6
    
elif option == 2:
    H = 4.1 #m
    Drod = 1.227/100 # cm to m
    pitch = 1.62/100# cm to m 
    DFuel = 1.04/100 # cm to m 
    gap_t= 0.010/100 # cm to m
    k_gap = 0.25 #W/mC
    k_fuel = 3.6 #W/mC
    k_clad = 21.5 #W/mC
    G = 2350 #kg/m2s
    qo = 605*100 #W/m
    Pz0 = 7.5 #MPa
    Tfz0 = 272 #C
    Tf0 = Tfz0+273.15
    g = 9.81 #m/s2
    mu = 97.352e-6
    mul = 89.451e-6
    muv = 11.656e-6

z = np.linspace(0,H,1000)

#GEOMETRY
Af = (pitch)**2-1/4*np.pi*Drod**2
xi_h = xi_w = np.pi*Drod
Dh = 4*Af/(xi_w)

#HEAT GENERATION AND TRANSFER
def qp(x):
    qp = qo*np.sin(x*np.pi/H)
    return qp

def qpp(x):
    qpp = qp(x)/xi_h
    return qpp

#PHYSICAL PROPERTIES
water = XSteam(XSteam.UNIT_SYSTEM_BARE) #for determining properties, need temp and pressure to be in K and MPa
tf0,p0 = Tf0,Pz0

cp = water.Cp_pt(p0,tf0)*1e3 #j/kg
k = water.tc_pt(p0,tf0)  # W/mK

#defining functions for hfg, hf, and tsat => all functions of pressure

def hf(x):  
    hf =  water.hL_p(x) * 1e3
    return hf

def hg(x):  
    hg =  water.hV_p(x) * 1e3
    return hg

def hfg(x):  
    hfg = (hg(x) - hf(x))
    return hfg
    
def tsat(x):  
    tsat = water.tsat_p(x)
    return tsat


#initializing equilibrium quality and density
#must determine whether fluid temp is at saturation at the very entrance
Xe0 = cp*(tf0 - tsat(p0))/hfg(p0)
rho0 = water.rho_pt(p0,tf0)

##DETERMINING DIMENSIONLESS GROUPS
Re = G*Dh/mu
Pr = cp*mu/k

# need to determine whether flow is turbelent or not
if Re > 10000:
    Nu = 0.023*Re**(0.8)*Pr**(0.4)
elif Re < 2100:
    Nu = 4.36
    
h_val = Nu*k/Dh
f = Re**(-0.25)*0.316

#Two-Phase Properties
def pratio(x):
    pratio = water.rhoV_p(x) / water.rhoL_p(x)
    return pratio

#pm-pf/(pg-pf)
def alpha(x, y): #x = Xe, y = P
    alpha = 1 / (1 + (x**(-1) - 1) * pratio(y))
    return alpha
    
pfg = water.rhoL_p(p0) - water.rhoV_p(p0)
pL = water.rhoL_p(p0)
pV = water.rhoV_p(p0)

#defining how enthalpy changes with respect to pressure
def dhfdp(x):
    dhfdp = (7.11762e-16*(x)**2-1.85296e-8*(x)+1.52758e-1) #J/kg/Pa
    return dhfdp

def dhfgdp(x):
    dhfgdp = (-7.7343e-16*(x)**2+1.65472e-8*(x)+1.4331e-1)
    return dhfgdp

#FINDING PRESSURE AND FLUID TEMPERATURE
P = [Pz0*1e6]
Xe = [Xe0]
X = [0]
Tf = [Tf0]
Tsat = [tsat(Pz0)]
for i in range(1, len(z)):
    #FINDING PRESSURE
    dz = z[i]- z[i-1]
    if Xe[i-1] <= 0:
        X[i-1] = 0
    elif Xe[i-1] < 1:
        X[i-1] = Xe[i-1]
    v_l = water.vL_p(Pz0)
    v_g = water.vV_p(Pz0)
    v_fg = v_g-v_l
    p_m = 1/(v_l+X[i-1]*(v_fg))

    if Xe[i-1]<=0:
        iP = (g/v_l+f*G**2*v_l*xi_h/(2*Af))
    elif Xe[i-1] < 1:
        mum = 1/(Xe[i-1]/muv + (1-Xe[i-1])/mul)
        f_val = 0.046*Re**(-0.2)*(mul/mum)**(-0.2)
        tP_1 = 1/2*xi_w/Af*f_val*G**2*(v_l+X[i-1]*(v_fg))
        tP_2 = p_m*g
        tP_3 = G*qp(z[i])*(v_g-v_l)/(Af*hfg(P[i-1]*1e-6))
        bP_1 = dhfdp(P[i-1])+Xe[i-1]*dhfgdp(P[i-1])
        bP_2 = 1-G**2*(v_g-v_l)/(hfg(P[i-1]*1e-6))*bP_1
        iP = (tP_1+tP_2+tP_3)/bP_2
    P_i = P[i-1]-dz*iP
    dpdz = (P_i-P[i-1])/dz
    #FINDING QUALITY NOW
    iXe_1 = dhfdp(P[i-1])*dpdz+Xe[i-1]*dhfgdp(P[i-1])*dpdz
    iXe_2 = qp(z[i])/(Af*G*hfg(P_i*1e-6))-1/hfg(P_i*1e-6)*iXe_1
    Xei = dz*iXe_2+Xe[i-1]

    #FINDING FLUID TEMPERATURE NOW
    if Xei <= 0:
        Tfi = hfg(P_i*1e-6)/cp*Xei+tsat(P_i*1e-6)
    else:
        Tfi = tsat(P_i*1e-6)
        
    Tsat.append(tsat(P_i*1e-6))    
    P.append(P_i)
    Xe.append(Xei)
    X.append(X)
    Tf.append(Tfi)

Tsat[-1], P[-1], Xe[-1], Tf[-1]

In [ ]:
#FINDING STEAM QUALITY
Xe_ar = np.array(Xe)
X_ar = np.zeros(z.size)
for i in range(len(z)):
    if Xe_ar[i] < 0:
        X_ar[i] = 1e-12
    else:
        X_ar[i] = Xe_ar[i]
        
#FINDING SURFACE TEMPERATURE
q_p = qo*np.sin(z*np.pi/H)
Tsat_ = []
for i in P:
    Tsat_.append(tsat(i*1e-6))
tsat_ar = np.array(Tsat_)
tw = np.zeros(z.size)
P_ar = np.array(P)*1e-6
z_ar = np.array(z)
tsat_ar = np.array(Tsat_)
tf_ar = np.array(Tf)
twall = np.zeros(z.size)
for i in range(z.size):
    twall = q_p[i]/(xi_h*h_val)+tf_ar[i]
    if (twall<=tsat_ar[i]):
        tw[i] = twall
    else:
        F=(1+X_ar[i]*Pr*(pL/pV - 1))**(0.35)
        S = 1/(1+0.055*F**0.1*Re**0.16)
        qpp = q_p[i]/xi_h
        prat = P_ar[i]/22.064
        hNB = 55*(prat)**0.12*18**(-0.5)*qpp**(2/3)*(-np.log10(prat))**(-0.55)
        
        #coefficients for quad eq
        c = -qpp**2+F**2*h_val**2*tf_ar[i]**2+S**2*hNB**2*tsat_ar[i]**2
        a = F**2*h_val**2+S**2*hNB**2
        b = -2*(F**2*h_val**2*tf_ar[i]+S**2*hNB**2*tsat_ar[i])

        deter = b**2-4*a*c
        tw[i] = (-b+np.sqrt(deter))/(2*a)

#with the determined Tw, conduction analysis employed to find the radial temperature components of fuel pin
#geometry of the fuel pin!!!!
Rf = DFuel /2
Rci = DFuel /2 + gap_t
Rco = Drod / 2
Aci = 2*Rci * np.pi
Af = 2*Rf * np.pi
Axf = Rf**2 * np.pi
        
#fuel integration constants
C1 = 0
C3 = -k_fuel/k_gap*(qp(z)*(Rf)**2/(2*k_fuel*Axf))
C5 = k_gap/k_clad*C3
C6 = tw - C5*np.log(Rco)
C4 = C5*np.log(Rci)-C3*np.log(Rci)+C6
C2 = C3*np.log(Rf)+C4+qp(z)*(Rf)**2/(4*k_fuel*Axf)

def T_fuel(x):
    T_fuel = -qp(z)*x**2/(4*k_fuel*Axf)+C2
    return T_fuel

def T_gap(x):
    T_gap = C3*np.log(x)+C4
    return T_gap

def T_clad(x):
    T_clad = C5*np.log(x)+C6
    return T_clad

tck = np.zeros(z.size)
for i in range(z.size):
    tck[i] = 273.15
tw_c = tw-tck

In [ ]:
#determining location of max centerline temperature for cladding, coolant, and fuel
#fliud max temp
print('fluid max temp:',max(Tf)-273.15)

#maximum fuel temp occurs at centerline:
print('fuel max temp:',max(T_fuel(0))-273.15)

#maximum cladding temp:
print('fuel clad temp:',max(T_clad(Rco))-273.15)

In [ ]:
#CHF, DNB, and CPR for both PWR and BWR

#before doing anything, need to find some knowns 
Xe_abs = [abs(x) for x in Xe]
Xe_abs = np.array(Xe_abs)

hin = []
for i in range(len(z)):
    hin.append(hf(p0))
hin_ar = np.array(hin)*1e-3
hf_ = []
for i in P_ar:
    hf_.append(hf(i))
hf_ar = np.array(hf_)*1e-3


#Finding critical heat flux that causes DNB using W-3 correlation
#now defining parts of the function for
first_part = (2.022-0.06238*P_ar)+(0.1722-0.01427*P_ar)*np.exp((18.177-0.5987*P_ar)*Xe_ar)
second_part = (0.1484-1.596*Xe_ar+0.1729*Xe_ar*Xe_abs)*2.326*G+3271
third_part = 1.157-0.869*Xe_ar
fourth_part = 0.2664+0.8357*np.exp(-124.1*Dh)
fifth_part = 0.8258+0.0003413*(hf_ar-hin_ar)
qcr_DNB = first_part*second_part*third_part*fourth_part*fifth_part

#dryout now!!!!!, only for BWR
#for this case, use the Bowring Correlation
#MAY NOT BE NEEDED
if option == 2:
    hfg_ = []
    for i in P_ar:
        hfg_.append(hfg(i))
    hfg_ar = np.array(hfg_)
    Pr = []
    for i in P:
        Pr.append(0.145*i*1e-6)
    Pr_ar = np.array(Pr)
    if min(Pr) < 1:
        F1 = (Pr_ar**18.942*np.exp(20.89*(1-Pr_ar))+0.917)/1.917
        F2 = F1/((Pr_ar**1.316*np.exp(2.444*(1-Pr_ar))+0.309)/1.309)
        F3 = (Pr_ar**17.023*np.exp(16.658*(1-Pr_ar))+0.667)/1.667
        F4 = F3*Pr_ar**1.649
    if min(Pr) > 1:
        F1 = Pr_ar**(-0.368)*np.exp(0.648*(1-Pr_ar))
        F2 = F1/(Pr_ar**(-0.448)*np.exp(0.245*(1-Pr_ar)))
        F3 = Pr_ar**0.219
        F4 = F3*Pr_ar**1.649
    n = 2.0-0.5*Pr_ar
    A = 2.317*(hfg_ar*Drod*G/4)*F1
    B = Drod*G/4
    C = 0.077*F3*Drod*G/(1+0.347*F4*(G/1356)**n)
    q_cr_dryout = (A-B*hfg_ar*X_ar)/C

if option == 1:
    plt.plot(qp(z)*1e-3/xi_h,z,label = 'Operational heat flux')
    plt.plot(qcr_DNB,z,label = 'DNB heat flux')
    plt.xlabel('Heat Flux [$W/m^2$]')
    plt.ylabel('Axial Position [m]')
    plt.title('DNB Heat Flux and Operational Heat Flux')
    plt.grid()
    plt.legend()
    plt.show()

    DNBR = qcr_DNB/(qp(z)*1e-3/xi_h)
    plt.plot(DNBR[30:980],z[30:980])
    plt.xlabel('Departure from Nucleate Boiling Ratio')
    plt.ylabel('Axial Position [m]')
    plt.title('Departure from Nucleate Boiling Ratio against Axial Position')
    plt.grid()
    plt.show()
    print(min(qcr_DNB/(qp(z)*1e-3/xi_h)))
    
if option == 2:
    plt.plot(qp(z)*1e-3/xi_h,z,label = 'Operational heat flux')
    plt.plot(qcr_DNB,z,label = 'DNB heat flux')
    plt.xlabel('Heat Flux [$W/m^2$]')
    plt.ylabel('Axial Position [m]')
    plt.title('DNB Heat Flux and Operational Heat Flux')
    plt.grid()
    plt.legend()
    plt.show()

    DNBR = qcr_DNB/(qp(z)*1e-3/xi_h)
    plt.plot(DNBR[30:980],z[30:980])
    plt.xlabel('Departure from Nucleate Boiling Ratio')
    plt.ylabel('Axial Position [m]')
    plt.title('Departure from Nucleate Boiling Ratio against Axial Position')
    plt.grid()
    plt.show()
    print(min(qcr_DNB/(qp(z)*1e-3/xi_h)))

In [ ]:
#NEED A NEW FUNCTION TO DEFINE THE CPR
#this function allows us to change the initial power of the linear generation rate!!!
def q_multi(n):
    H = 4.1 #m
    Drod = 1.227/100 # cm to m
    pitch = 1.62/100# cm to m 
    DFuel = 1.04/100 # cm to m 
    gap_t= 0.010/100 # cm to m
    k_gap = 0.25 #W/mC
    k_fuel = 3.6 #W/mC
    k_clad = 21.5 #W/mC
    G = 2350 #kg/m2s
    qo = n*605*100 #W/m
    Pz0 = 7.5 #MPa
    Tfz0 = 272 #C
    Tf0 = Tfz0+273.15
    g = 9.81 #m/s2
    mu = 97.352e-6
    mul = 89.451e-6
    muv = 11.656e-6
    
    z = np.linspace(0,H,1000)
    
    #GEOMETRY
    Af = (pitch)**2-1/4*np.pi*Drod**2
    xi_h = xi_w = np.pi*Drod
    Dh = 4*Af/(xi_w)
    
    #HEAT GENERATION AND TRANSFER
    def qp(x):
        qp = qo*np.sin(x*np.pi/H)
        return qp
    
    def qpp(x):
        qpp = qp(x)/xi_h
        return qpp
    
    #PHYSICAL PROPERTIES
    water = XSteam(XSteam.UNIT_SYSTEM_BARE) #for determining properties, need temp and pressure to be in K and MPa
    tf0,p0 = Tf0,Pz0
    
    cp = water.Cp_pt(p0,tf0)*1e3 #j/kg
    k = water.tc_pt(p0,tf0)  # W/mK
    
    #defining functions for hfg, hf, and tsat => all functions of pressure
    
    def hf(x):  
        hf =  water.hL_p(x) * 1e3
        return hf
    
    def hg(x):  
        hg =  water.hV_p(x) * 1e3
        return hg
    
    def hfg(x):  
        hfg = (hg(x) - hf(x))
        return hfg
        
    def tsat(x):  
        tsat = water.tsat_p(x)
        return tsat
    
    
    #initializing equilibrium quality and density
    #must determine whether fluid temp is at saturation at the very entrance
    Xe0 = cp*(tf0 - tsat(p0))/hfg(p0)
    rho0 = water.rho_pt(p0,tf0)
    
    ##DETERMINING DIMENSIONLESS GROUPS
    Re = G*Dh/mu
    Pr = cp*mu/k
    
    # need to determine whether flow is turbelent or not
    if Re > 10000:
        Nu = 0.023*Re**(0.8)*Pr**(0.4)
    elif Re < 2100:
        Nu = 4.36
        
    h_val = Nu*k/Dh
    f = Re**(-0.25)*0.316
    
    #Two-Phase Properties
    def pratio(x):
        pratio = water.rhoV_p(x) / water.rhoL_p(x)
        return pratio
    
    #pm-pf/(pg-pf)
    def alpha(x, y): #x = Xe, y = P
        alpha = 1 / (1 + (x**(-1) - 1) * pratio(y))
        return alpha
        
    pfg = water.rhoL_p(p0) - water.rhoV_p(p0)
    pL = water.rhoL_p(p0)
    pV = water.rhoV_p(p0)
    
    #defining how enthalpy changes with respect to pressure
    def dhfdp(x):
        dhfdp = (7.11762e-16*(x)**2-1.85296e-8*(x)+1.52758e-1) #J/kg/Pa
        return dhfdp
    
    def dhfgdp(x):
        dhfgdp = (-7.7343e-16*(x)**2+1.65472e-8*(x)+1.4331e-1)
        return dhfgdp
    
    #FINDING PRESSURE AND FLUID TEMPERATURE
    P = [Pz0*1e6]
    Xe = [Xe0]
    X = [0]
    Tf = [Tf0]
    Tsat = [tsat(Pz0)]
    for i in range(1, len(z)):
        #FINDING PRESSURE
        dz = z[i]- z[i-1]
        if Xe[i-1] <= 0:
            X[i-1] = 0
        elif Xe[i-1] < 1:
            X[i-1] = Xe[i-1]
        v_l = water.vL_p(Pz0)
        v_g = water.vV_p(Pz0)
        v_fg = v_g-v_l
        p_m = 1/(v_l+X[i-1]*(v_fg))
    
        if Xe[i-1]<=0:
            iP = (g/v_l+f*G**2*v_l*xi_h/(2*Af))
        elif Xe[i-1] < 1:
            mum = 1/(Xe[i-1]/muv + (1-Xe[i-1])/mul)
            f_val = 0.046*Re**(-0.2)*(mul/mum)**(-0.2)
            tP_1 = 1/2*xi_w/Af*f_val*G**2*(v_l+X[i-1]*(v_fg))
            tP_2 = p_m*g
            tP_3 = G*qp(z[i])*(v_g-v_l)/(Af*hfg(P[i-1]*1e-6))
            bP_1 = dhfdp(P[i-1])+Xe[i-1]*dhfgdp(P[i-1])
            bP_2 = 1-G**2*(v_g-v_l)/(hfg(P[i-1]*1e-6))*bP_1
            iP = (tP_1+tP_2+tP_3)/bP_2
        P_i = P[i-1]-dz*iP
        dpdz = (P_i-P[i-1])/dz
        #FINDING QUALITY NOW
        iXe_1 = dhfdp(P[i-1])*dpdz+Xe[i-1]*dhfgdp(P[i-1])*dpdz
        iXe_2 = qp(z[i])/(Af*G*hfg(P_i*1e-6))-1/hfg(P_i*1e-6)*iXe_1
        Xei = dz*iXe_2+Xe[i-1]
    
        #FINDING FLUID TEMPERATURE NOW
        if Xei < 0:
            Tfi = hfg(P_i*1e-6)/cp*Xei+tsat(P_i*1e-6)
        else:
            Tfi = tsat(P_i*1e-6)
            
        Tsat.append(tsat(P_i*1e-6))    
        P.append(P_i)
        Xe.append(Xei)
        X.append(X)
        Tf.append(Tfi)
    
    #FINDING STEAM QUALITY
    Xe_ar = np.array(Xe)
    X_ar = np.zeros(z.size)
    for i in range(len(z)):
        if Xe_ar[i] < 0:
            X_ar[i] = 1e-12
        else:
            X_ar[i] = Xe_ar[i]
            
    #FINDING SURFACE TEMPERATURE
    q_p = qo*np.sin(z*np.pi/H)
    Tsat_ = []
    for i in P:
        Tsat_.append(tsat(i*1e-6))
    tsat_ar = np.array(Tsat_)
    tw = np.zeros(z.size)
    P_ar = np.array(P)*1e-6
    z_ar = np.array(z)
    tsat_ar = np.array(Tsat_)
    tf_ar = np.array(Tf)
    twall = np.zeros(z.size)
    for i in range(z.size):
        twall = q_p[i]/(xi_h*h_val)+tf_ar[i]
        if (twall<=tsat_ar[i]):
            tw[i] = twall
        else:
            F=(1+X_ar[i]*Pr*(pL/pV - 1))**(0.35)
            S = 1/(1+0.055*F**0.1*Re**0.16)
            qpp = q_p[i]/xi_h
            prat = P_ar[i]/22.064
            hNB = 55*(prat)**0.12*18**(-0.5)*qpp**(2/3)*(-np.log10(prat))**(-0.55)
            
            #coefficients for quad eq
            c = -qpp**2+F**2*h_val**2*tf_ar[i]**2+S**2*hNB**2*tsat_ar[i]**2
            a = F**2*h_val**2+S**2*hNB**2
            b = -2*(F**2*h_val**2*tf_ar[i]+S**2*hNB**2*tsat_ar[i])
    
            deter = b**2-4*a*c
            tw[i] = (-b+np.sqrt(deter))/(2*a)
    
    #with the determined Tw, conduction analysis employed to find the radial temperature components of fuel pin
    #geometry of the fuel pin!!!!
    Rf = DFuel /2
    Rci = DFuel /2 + gap_t
    Rco = Drod / 2
    Aci = 2*Rci * np.pi
    Af = 2*Rf * np.pi
    Axf = Rf**2 * np.pi
            
    #fuel integration constants
    C1 = 0
    C3 = -k_fuel/k_gap*(qp(z)*(Rf)**2/(2*k_fuel*Axf))
    C5 = k_gap/k_clad*C3
    C6 = tw - C5*np.log(Rco)
    C4 = C5*np.log(Rci)-C3*np.log(Rci)+C6
    C2 = C3*np.log(Rf)+C4+qp(z)*(Rf)**2/(4*k_fuel*Axf)
    
    def T_fuel(x):
        T_fuel = -qp(z)*x**2/(4*k_fuel*Axf)+C1*x+C2
        return T_fuel
    
    def T_gap(x):
        T_gap = C3*np.log(x)+C4
        return T_gap
    
    def T_clad(x):
        T_clad = C5*np.log(x)+C5
        return T_clad
    
    tck = np.zeros(z.size)
    for i in range(z.size):
        tck[i] = 273.15
    return np.array(Xe),z,np.array(P)*1e-6